# OBTENCIÓN DISTANCIAS

En este *notebook*, se muestra como calcular las matrices de tiempo y distancia entre estaciones utilizando la versión gratuita de la API de Google Maps.

In [ ]:
import requests
import numpy as np
import pandas as pd
import time

Para el cálculo, se necesita cargar un conjunto de datos con esta estructura:

In [6]:
datos.head(5)

,station_id,lat,lon
0,1.0,41.397978,2.180107
1,2.0,41.395488,2.177198
2,3.0,41.394156,2.181331
3,4.0,41.393317,2.181248
4,5.0,41.391103,2.180176


In [ ]:
stations = datos.copy()

# API Key (sustitúyela por tu propia clave)
api_key = ""

# Generar lista de ubicaciones
locations = [f"{row['lat']},{row['lon']}" for _, row in stations.iterrows()]

# Crear matrices vacías para distancias y tiempos
n = len(stations)
distance_matrix = np.zeros((n, n))
time_matrix = np.zeros((n, n))

# URL base para la API de Google
base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"

# Función para dividir la lista en partes más pequeñas (máximo 10 orígenes y 10 destinos para evitar el límite de la API)
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Máximo permitido de elementos por solicitud (100 elementos, donde elementos = orígenes * destinos)
max_origins_destinations = 10 

# Hacer la consulta para cada grupo de estaciones
for i_chunk in chunk_list(range(n), max_origins_destinations):
    print(i_chunk)
    origins = [locations[i] for i in i_chunk]
    
    for j_chunk in chunk_list(range(n), max_origins_destinations):
        destinations = [locations[j] for j in j_chunk]
        
        # Parámetros para la solicitud
        params = {
            "origins": "|".join(origins),
            "destinations": "|".join(destinations),
            "mode": "driving",  
            "departure_time": 'now',  
            "key": api_key
        }
        
        # Realizar la solicitud
        response = requests.get(base_url, params=params)
        result = response.json()

        # Manejo de posibles errores en la respuesta
        if result.get('status') == 'OK':
            rows = result['rows']
            for i_index, row in enumerate(rows):
                elements = row['elements']
                for j_index, element in enumerate(elements):
                    if element['status'] == 'OK':
                        distance = element['distance']['value']  # en metros
                        duration = element['duration']['value']  # en segundos
                        distance_matrix[i_chunk[i_index], j_chunk[j_index]] = distance
                        time_matrix[i_chunk[i_index], j_chunk[j_index]] = duration
                    else:
                        print(f"Error en el cálculo entre {origins[i_index]} y {destinations[j_index]}: {element['status']}")
        else:
            print(f"Error en la respuesta: {result.get('status')}")
        
        # Agregar un pequeño retraso para evitar exceder los límites de la API
        time.sleep(1)

# Mostrar matrices
station_ids = stations['station_id'].tolist()

# Crear los DataFrames con los IDs correctos como índices y columnas
distance_df = pd.DataFrame(distance_matrix, columns=station_ids, index=station_ids)
time_df = pd.DataFrame(time_matrix, columns=station_ids, index=station_ids)

# Mostrar los resultados
print("Matriz de Distancias (en metros):")
print(distance_df)

print("\nMatriz de Tiempos (en segundos):")
print(time_df)

In [ ]:
distance_df.to_csv('../../1-DATOS/2-DATOS PROCESADOS/DISTANCIAS/Matriz de distancias.csv')

In [ ]:
time_df.to_csv('../../1-DATOS/2-DATOS PROCESADOS/DISTANCIAS/Matriz de tiempos.csv')